In [12]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# 모델 저장 폴더 설정
save_dir = '/home/heechun/final_ws/TableCheck/1114_final_table_check_2'
os.makedirs(save_dir, exist_ok=True)  # 폴더가 없을 경우 생성

# 데이터 증강을 위한 ImageDataGenerator 설정 (강화된 옵션 포함)
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,              
    rotation_range=40,               
    width_shift_range=0.4,          
    height_shift_range=0.4,          
    shear_range=0.3,                 
    zoom_range=[0.8, 1.5],          
    horizontal_flip=True,            
    brightness_range=[0.7, 1.3],    
    channel_shift_range=50.0,       
    fill_mode='nearest',             
    validation_split=0.2             
)

# 학습용 데이터 로드
train_generator = train_datagen.flow_from_directory(
    '/home/heechun/final_ws/TableCheck/record_table/data/train/',
    target_size=(224, 224),          
    batch_size=32,                   
    class_mode='binary',             
    subset='training'                
)

# 검증용 데이터 로드
validation_generator = train_datagen.flow_from_directory(
    '/home/heechun/final_ws/TableCheck/record_table/data/train/',
    target_size=(224, 224),          
    batch_size=32,                   
    class_mode='binary',             
    subset='validation'              
)

# MobileNetV2 기반 모델 구성
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

# 특정 레이어까지만 고정하고 나머지 레이어를 미세 조정하도록 설정
fine_tune_at = 30
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# 모델 정의 (Dense 레이어 개선 및 L2 정규화 추가)
model = tf.keras.models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss='binary_crossentropy', metrics=['accuracy'])

# 체크포인트 설정
checkpoint_path = os.path.join(save_dir, 'final_best_model_1114_2.keras')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1
)

# Early Stopping 설정
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=15, restore_best_weights=True, verbose=1
)

# 학습률 감소 스케줄러 설정
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, verbose=1
)

# 학습 로그 저장 (TensorBoard 사용)
log_dir = os.path.join(save_dir, 'logs_1114_2')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

# 모델 학습
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=80,
    callbacks=[model_checkpoint, tensorboard_callback, early_stopping, lr_scheduler]
)


Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/80


/home/heechun/venv/md_ven/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.5252 - loss: 7.6367
Epoch 1: val_accuracy improved from -inf to 0.69500, saving model to /home/heechun/final_ws/TableCheck/1114_final_table_check_2/final_best_model_1114_2.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 40s 571ms/step - accuracy: 0.5264 - loss: 7.6345 - val_accuracy: 0.6950 - val_loss: 7.3821 - learning_rate: 5.0000e-05
Epoch 2/80
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step - accuracy: 0.5979 - loss: 7.4741
Epoch 2: val_accuracy improved from 0.69500 to 0.78250, saving model to /home/heechun/final_ws/TableCheck/1114_final_table_check_2/final_best_model_1114_2.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 499ms/step - accuracy: 0.5992 - loss: 7.4717 - val_accuracy: 0.7825 - val_loss: 7.2614 - learning_rate: 5.0000e-05
Epoch 3/80
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step - accuracy: 0.6909 - loss: 7.2666
Epoch 3: val_accuracy improved from 0.78250 to 0.81000, saving model to /home/heechun/final_ws/TableCheck/1114_final_table_check_2/final_best_

In [13]:
!tensorboard --logdir=/home/heechun/final_ws/TableCheck/1114_final_table_check_2/logs_1114_2

2024-11-14 16:32:04.665521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-14 16:32:04.681158: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-14 16:32:04.686495: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 16:32:04.697611: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-14 16:32:05.488092: W tensorflow/compiler/tf2